In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
from typing import Optional
from sklearn.preprocessing import MinMaxScaler
from dataset.data import *
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
class StockDataset(Dataset):
    def __init__(self, stock_data, sequence_length=30):
        """
        Args:
            stock_data (list): List of dictionaries containing 'history' and 'target'.
            sequence_length (int): Number of historical days used as input.
        """
        self.data = stock_data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx]['history'], dtype=torch.float32).unsqueeze(-1)
        y = torch.tensor(self.data[idx]['target'], dtype=torch.float32)
        return x, y

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, output_size: int, dropout: float = 0.2):
        """
        LSTM-based model for time series prediction.

        Args:
            input_size (int): Number of features per time step.
            hidden_size (int): Number of hidden units in LSTM layers.
            num_layers (int): Number of LSTM layers.
            output_size (int): Number of output features.
            dropout (float): Dropout rate.
        """
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
def eval_lstm(model, test_loader, device):
    """
    Evaluate the LSTM model.
    """
    model.eval()
    total_mape = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            out = model(inputs)
            mape = torch.mean(torch.abs((targets - out.squeeze()) / (targets + 1e-7)))
            total_mape += mape.item()

    return total_mape / len(test_loader)

In [ ]:
def eval_lstm_rmse(model, test_loader, device):
    """
    Evaluate the LSTM model using RMSE (Root Mean Squared Error).
    """
    model.eval()
    total_rmse = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            out = model(inputs)
            rmse = torch.sqrt(torch.mean((targets - out.squeeze())**2))
            total_rmse += rmse.item()

    return total_rmse / len(test_loader)

In [ ]:
def train_lstm(model, optimizer, train_loader, test_loader, num_epochs, device):
    """
    Train the LSTM model with proper mode management.
    """
    model.train()
    average_train_loss = []
    average_test_mape = []

    for epoch in tqdm(range(num_epochs)):
        total_train_loss = 0

        # Training loop
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            out = model(inputs)

            loss = F.mse_loss(out.squeeze(), targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        average_loss = total_train_loss / len(train_loader)
        average_train_loss.append(average_loss)

        # Evaluation loop
        test_mape = eval_lstm(model, test_loader, device)
        average_test_mape.append(test_mape)

        if epoch % 149 == 0:
            print(f"Epoch {epoch:03d}, Train Loss: {average_loss:.4f}, Val MAPE: {test_mape:.4f}")

    return average_train_loss, average_test_mape

In [ ]:
def prepare_stock_data_by_stock(dataset, sequence_length=30):
    """
    Prepares the stock data for LSTM training by keeping each stock's data separate.

    Args:
        dataset (NASDAQDataset): The initialized and processed NASDAQDataset object.
        sequence_length (int): Number of historical days to use as input.

    Returns:
        dict: A dictionary where keys are stock tickers and values are lists of samples.
    """
    stock_data_by_stock = {}

    for stock in dataset.filtered_stock_list:
        stock_df = dataset.data[stock]
        stock_data = []
        for i in range(sequence_length, len(stock_df) - 1):
            history = stock_df['Adj Close'].iloc[i - sequence_length:i].values
            target = stock_df['Adj Close'].iloc[i + 1]
            stock_data.append({
                "history": history.tolist(),
                "target": target
            })
        stock_data_by_stock[stock] = stock_data

    return stock_data_by_stock

In [ ]:
def main():
    # Initialize dataset
    stocks_path = "dataset\\dataset\\stocks"
    meta_path = "dataset\\dataset\\symbols_valid_meta.csv"
    nasdaq_100_path = "dataset\\nasdaq_100.csv"
    start_date = "2014-01-01"

    dataset = NASDAQDataset(stocks_path, meta_path, nasdaq_100_path, start_date)
    dataset.read_data()

    # Prepare stock data for LSTM
    sequence_length = 10
    stock_data_by_stock = prepare_stock_data_by_stock(dataset, sequence_length)

    # Training parameters
    input_size = 1
    hidden_size = 128
    num_layers = 3
    output_size = 1
    epochs = 150
    batch_size = 16
    learning_rate = 0.001

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    all_test_mape = []
    all_test_rmse = []

    for stock, stock_data in stock_data_by_stock.items():
        print(f"Training model for stock: {stock}")

        # Split into train and test sets
        train_size = int(0.8 * len(stock_data))
        train_data = stock_data[:train_size]
        test_data = stock_data[train_size:]

        train_dataset = StockDataset(train_data, sequence_length)
        test_dataset = StockDataset(test_data, sequence_length)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Initialize LSTM model for this stock
        model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout=0.3).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

        # Train the model
        _, test_mape = train_lstm(model, optimizer, train_loader, test_loader, epochs, device)
        all_test_mape.append(test_mape[-1])

        # Evaluate RMSE
        rmse = eval_lstm_rmse(model, test_loader, device)
        all_test_rmse.append(rmse)

    # Average evaluation metrics
    average_mape = sum(all_test_mape) / len(all_test_mape)
    average_rmse = sum(all_test_rmse) / len(all_test_rmse)
    print(f"Average MAPE across all stocks: {average_mape:.4f}")
    print(f"Average RMSE across all stocks: {average_rmse:.4f}")

    # Optionally, save per-stock metrics
    for stock, mape, rmse in zip(stock_data_by_stock.keys(), all_test_mape, all_test_rmse):
        print(f"Stock: {stock}, Final MAPE: {mape:.4f}, Final RMSE: {rmse:.4f}")


In [10]:
main()

  0%|          | 0/101 [00:00<?, ?it/s]

Data for APP is missing!
Data for ARM is missing!


 39%|███▊      | 39/101 [00:00<00:00, 118.02it/s]

Data for GFS is missing!
Data for META is missing!


 74%|███████▍  | 75/101 [00:00<00:00, 112.48it/s]

...

Epoch 149, Train Loss: 18.4833, Val MAPE: 0.1293
Training model for stock: ADSK


  1%|          | 1/150 [00:00<00:43,  3.40it/s]

Epoch 000, Train Loss: 6273.9647, Val MAPE: 0.9012


100%|██████████| 150/150 [00:38<00:00,  3.90it/s]

Epoch 149, Train Loss: 10.2735, Val MAPE: 0.0871
Average MAPE across all stocks: 0.1193
Average RMSE across all stocks: 48.3603
Stock: LIN, Final MAPE: 0.1514, Final RMSE: 30.1868
Stock: AZN, Final MAPE: 0.1056, Final RMSE: 4.9626
Stock: BKR, Final MAPE: 0.1157, Final RMSE: 2.0052
Stock: AVGO, Final MAPE: 0.1266, Final RMSE: 37.4761
Stock: BIIB, Final MAPE: 0.0491, Final RMSE: 14.9145
Stock: BKNG, Final MAPE: 0.2875, Final RMSE: 546.7792
Stock: CDNS, Final MAPE: 0.2953, Final RMSE: 20.1834
Stock: ADBE, Final MAPE: 0.1030, Final RMSE: 33.6612
Stock: CHTR, Final MAPE: 0.0903, Final RMSE: 44.2028
Stock: CPRT, Final MAPE: 0.1460, Final RMSE: 12.7865
Stock: CSGP, Final MAPE: 0.2258, Final RMSE: 135.7871
Stock: CTAS, Final MAPE: 0.1424, Final RMSE: 37.4239
Stock: CSCO, Final MAPE: 0.0782, Final RMSE: 4.2303
Stock: CMCSA, Final MAPE: 0.0636, Final RMSE: 2.8705
Stock: COST, Final MAPE: 0.1503, Final RMSE: 43.7759
Stock: CSX, Final MAPE: 0.0331, Final RMSE: 2.6817
Stock: CTSH, Final MAPE: 0.022